In [1]:
import sys
#for creating the mapper code
from sqlalchemy import Column, ForeignKey, Integer, String, Float

#for configuration and class code
from sqlalchemy.ext.declarative import declarative_base

#for creating foreign key relationship between the tables
from sqlalchemy.orm import relationship

#for configuration
from sqlalchemy import create_engine

#create declarative_base instance
Base = declarative_base()

#we'll add classes here

#creates a create_engine instance at the bottom of the file
engine = create_engine('sqlite:///transcriptomics.db')

# Base.metadata.create_all(engine)

In [53]:
from sqlalchemy.orm import load_only

In [2]:
import GEOparse
import re
import os
import pandas as pd
import numpy as np
from scipy import stats
from GSEpipeline import *

# find project root dir
currentdir = os.getcwd()
dirlist = currentdir.split('/')
projectdir = '/'.join(dirlist[0:-1])

In [3]:
class Sample(Base):
    __tablename__ = 'sample'

    id = Column(Integer, primary_key=True)
    ENTREZ_GENE_ID = Column(String(250), nullable=False)
    VALUE = Column(Float, nullable=False)
    P_VALUE = Column(Float)
    ABS_CALL = Column(String(2))
    Sample = Column(String(64))
    
    
class IDMapps(Base):
    __tablename__ = 'id_entrez_map'
    
    idx = Column(Integer, primary_key=True)
    ID = Column(String(250))
    ENTREZ = Column(String(250))

class GSEinfo(Base):
    __tablename__ = 'gseinfo'
    
    Sample = Column(String(64), primary_key=True)
    GSE = Column(String(64))
    Platform = Column(String(64))

In [4]:
Base.metadata.create_all(engine)

In [5]:
# create instance of project
gse2770 = GSEproject('GSE2770',projectdir)

14-Aug-2019 13:34:45 INFO GEOparse - Parsing ./GSE2770_family.soft.gz: 
14-Aug-2019 13:34:45 DEBUG GEOparse - DATABASE: GeoMiame
14-Aug-2019 13:34:45 DEBUG GEOparse - SERIES: GSE2770
14-Aug-2019 13:34:45 DEBUG GEOparse - PLATFORM: GPL96


Initialize project (GSE2770):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW


14-Aug-2019 13:34:47 DEBUG GEOparse - PLATFORM: GPL97
14-Aug-2019 13:34:48 DEBUG GEOparse - PLATFORM: GPL8300
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60348
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60349
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60350
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60351
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60352
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60353
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60354
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60355
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60356
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60357
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60358
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60359
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60360
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60361
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60362
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60363
14-Aug-201

GPL96
[HG-U133A] Affymetrix Human Genome U133A Array
GPL97
[HG-U133B] Affymetrix Human Genome U133B Array
GPL8300
[HG_U95Av2] Affymetrix Human Genome U95 Version 2 Array


In [6]:
df_clean = gse2770.get_entrez_table_pipeline(fromcsv=True)
df_clean.sort_index(inplace=True)
df_clean

/usr/local/Cellar/ipython/7.5.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197,200) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,ENTREZ_GENE_ID,gsm60349.cel.gz,gsm60349.cel.gz.1,gsm60349.cel.gz.2,gsm60350.cel.gz,gsm60350.cel.gz.1,gsm60350.cel.gz.2,gsm60351.cel.gz,gsm60351.cel.gz.1,gsm60351.cel.gz.2,...,gsm60763.cel.gz.2,gsm60764.cel.gz,gsm60764.cel.gz.1,gsm60764.cel.gz.2,gsm60765.cel.gz,gsm60765.cel.gz.1,gsm60765.cel.gz.2,gsm60766.cel.gz,gsm60766.cel.gz.1,gsm60766.cel.gz.2
0,780 /// 100616237,717.599098,P,0.017000,966.038612,P,0.003067,724.167413,P,0.005643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5982,140.340345,A,0.204022,124.764102,A,0.107301,90.986765,A,0.250724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3310,68.989968,A,0.458816,55.205940,A,0.302547,97.283915,A,0.175989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7849,1281.802247,P,0.017000,902.213482,P,0.019303,919.399037,P,0.027860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2978,200.536466,A,0.218983,277.363337,A,0.060419,300.050853,A,0.117161,...,0.143002,373.110629,A,0.164032,626.187096,A,0.186972,401.599221,A,0.186972
5,7318 /// 100847079,540.953164,P,0.021866,255.370653,P,0.031336,506.635753,P,0.013092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7067,223.734251,P,0.027860,158.550443,P,0.021866,189.557546,A,0.189687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11099,65.106109,A,0.541184,64.526428,A,0.602006,49.510810,A,0.602006,...,0.570568,90.043163,A,0.535424,114.817064,A,0.605162,89.731430,A,0.638946
8,6352,217.342576,P,0.009985,285.981770,P,0.000388,214.198185,P,0.035163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1571,126.325873,P,0.039365,53.677210,A,0.418069,55.656157,A,0.267463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Write to DATABASE

In [7]:
df_samples = pd.DataFrame([],columns=['ENTREZ_GENE_ID','VALUE','P_VALUE','ABS_CALL','Sample'])
df_samples.index.name = 'id'

for key,val in gse2770.gsm_platform.items():
    col_val = '{}.cel.gz'.format(key.lower())
    col_abs = '{}.cel.gz.1'.format(key.lower())
    col_p = '{}.cel.gz.2'.format(key.lower())
    if not col_val in list(df_clean):
        continue
    df_s = pd.DataFrame([])
    df_s.index.name='id'
    df_s['ENTREZ_GENE_ID'] = df_clean['ENTREZ_GENE_ID']
    df_s['VALUE'] = df_clean[col_val]
    df_s['ABS_CALL'] = df_clean[col_abs]
    df_s['P_VALUE'] = df_clean[col_p]
    df_s['Sample'] = key.upper()
    df_samples = pd.concat([df_samples, df_s.dropna(how='any')], ignore_index=True)

df_gseinfo = pd.DataFrame([],columns=['Sample','GSE','Platform'])
df_gseinfo['Sample'] = pd.Series(list(gse2770.gsm_platform.keys()))
df_gseinfo['Platform'] = pd.Series(list(gse2770.gsm_platform.values()))
df_gseinfo['GSE'] = gse2770.gsename
# df_database.dropna(how='any',inplace=True)
# df_database.reset_index(inplace=True)

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [10]:
df_samples.to_sql(con=engine,name='sample',if_exists='replace',index_label='id')

In [14]:
df_gseinfo.to_sql(con=engine,name='gseinfo',if_exists='replace',index=False)

In [15]:
df = pd.read_sql(session.query(GSEinfo).filter().statement, session.bind)
df

,Sample,GSE,Platform
0,GSM60348,GSE2770,GPL8300
1,GSM60349,GSE2770,GPL8300
2,GSM60350,GSE2770,GPL8300
3,GSM60351,GSE2770,GPL8300
4,GSM60352,GSE2770,GPL8300
5,GSM60353,GSE2770,GPL8300
6,GSM60354,GSE2770,GPL8300
7,GSM60355,GSE2770,GPL8300
8,GSM60356,GSE2770,GPL8300
9,GSM60357,GSE2770,GPL8300


In [11]:
from sqlalchemy.orm import sessionmaker

DBSession = sessionmaker(bind=engine)

session = DBSession()

In [12]:
df = pd.read_sql(session.query(Sample).filter().statement, session.bind, index_col='id')
df

,ENTREZ_GENE_ID,VALUE,P_VALUE,ABS_CALL,Sample
id,,,,,
0,780 /// 100616237,717.599098,0.017000,P,GSM60349
1,5982,140.340345,0.204022,A,GSM60349
2,3310,68.989968,0.458816,A,GSM60349
3,7849,1281.802247,0.017000,P,GSM60349
4,2978,200.536466,0.218983,A,GSM60349
5,7318 /// 100847079,540.953164,0.021866,P,GSM60349
6,7067,223.734251,0.027860,P,GSM60349
7,11099,65.106109,0.541184,A,GSM60349
8,6352,217.342576,0.009985,P,GSM60349


In [32]:
aaa = session.query(Book).all()
aaa

 ...]

In [9]:
df_gseinfo

,Sample,GSE,Platform
0,GSM60348,GSE2770,GPL8300
1,GSM60349,GSE2770,GPL8300
2,GSM60350,GSE2770,GPL8300
3,GSM60351,GSE2770,GPL8300
4,GSM60352,GSE2770,GPL8300
5,GSM60353,GSE2770,GPL8300
6,GSM60354,GSE2770,GPL8300
7,GSM60355,GSE2770,GPL8300
8,GSM60356,GSE2770,GPL8300
9,GSM60357,GSE2770,GPL8300


In [35]:
# function to complete the inquery of a sheet
def queryTest(gse_ids,gsm_ids):
    print('Start Collecting Data for:')
    print(gse_ids)
    print(gsm_ids)
    

In [42]:
# input from user
filename = 'GeneExpressionDataUsed.xlsx'
sheet_name = list(range(5)) # first 5 sheets

inqueryFullPath = os.path.join(gse2770.datadir, filename)
inqueries = pd.read_excel(inqueryFullPath, sheet_name=sheet_name, header=0)

for i in range(2):
    # print(list(inqueries[i]))
    df = inqueries[i]
    sr = df['GSE ID'].dropna()
    gse_ids = sr[sr.str.match('GSE')].unique()
    sr = df['Samples'].dropna()
    gsm_ids = sr.unique()
    queryTest(gse_ids,gsm_ids)

Start Collecting Data for:
['GSE22886' 'GSE43005' 'GSE22045' 'GSE24634']
['GSM565273' 'GSM565274' 'GSM565275' 'GSM565290' 'GSM565291' 'GSM565292'
 'GSM1054773' 'GSM1054779' 'GSM1054781' 'GSM1054789' 'GSM548000'
 'GSM548001' 'GSM607510' 'GSM607511' 'GSM607512']
Start Collecting Data for:
['GSE2770' 'GSE22886' 'GSE43769' 'GSE43005' 'GSE50175' 'GSE30664']
['GSM60353' 'GSM60354' 'GSM60360' 'GSM60367' 'GSM60377' 'GSM60381'
 'GSM60738' 'GSM60739' 'GSM60745' 'GSM60752' 'GSM60762' 'GSM60766'
 'GSM60704' 'GSM60705' 'GSM60711' 'GSM60718' 'GSM60728' 'GSM60732'
 'GSM565993' 'GSM565994' 'GSM565995' 'GSM565996' 'GSM565997' 'GSM565276'
 'GSM565277' 'GSM565278' 'GSM565279' 'GSM565280' 'GSM1070758' 'GSM1070754'
 'GSM1070762' 'GSM1054775' 'GSM1054783' 'GSM1054787' 'GSM1054791'
 'GSM1215107' 'GSM1215109' 'GSM1215111' 'GSM1215113' 'GSM760599'
 'GSM760601']


In [178]:
df_results = pd.DataFrame([],columns=['ENTREZ_GENE_ID'])
df_results.set_index('ENTREZ_GENE_ID',inplace=True)
for gsm in gsm_ids:
    #print(gsm)
    df = pd.read_sql(session.query(Sample).filter_by(Sample=gsm).options(load_only("ABS_CALL","ENTREZ_GENE_ID")).statement, 
                     session.bind, 
                     index_col='ENTREZ_GENE_ID')
    df.drop('id',axis=1,inplace=True)
    df.rename(columns={'ABS_CALL':gsm},inplace=True)
    df.loc[df[gsm]=='A',gsm] = 0
    df.loc[df[gsm]=='P',gsm] = 1
    df.loc[df[gsm]=='M',gsm] = 1
    
    df_results = pd.concat([df_results, df], axis=1, sort=False)

# Need to set index name after merge
df_results.index.name = 'ENTREZ_GENE_ID'
df_results

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3310,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7849,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7067,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6352,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# df_results[['///' in s for s in df_results.index]]
df_results[df_results.index.str.contains('///')]
# df_results

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2342 /// 100529261,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6158 /// 102465483,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6144 /// 26771 /// 619499 /// 100131205,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3178 /// 664709,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6207 /// 85388 /// 100508408,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6205 /// 84546,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1655 /// 100616387 /// 100616408,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# df = pd.read_sql(session.query(GSEinfo).filter_by(Sample='GSM60728').statement, session.bind)
# print(df)
# df = pd.read_sql(session.query(Sample).filter_by(Sample='GSM60728').statement, 
#                  session.bind, 
#                  index_col='ENTREZ_GENE_ID',
#                  columns=['ABS_CALL']
#                 )
df = pd.read_sql(session.query(Sample).filter_by(Sample='GSM60728').options(load_only("ABS_CALL","ENTREZ_GENE_ID")).statement, 
                 session.bind, 
                 index_col='ENTREZ_GENE_ID'
                )
df.drop('id',axis=1,inplace=True)
print(df)

                                                 ABS_CALL
ENTREZ_GENE_ID                                           
780 /// 100616237                                       P
5982                                                    A
3310                                                    A
7849                                                    P
2978                                                    A
7318 /// 100847079                                      A
7067                                                    P
11099                                                   A
6352                                                    P
1571                                                    A
2049                                                    A
2101                                                    P
1548                                                    A
2621                                                    A
4323                                                    P
8717          

In [75]:
import mygene
mg = mygene.MyGeneInfo()
ens = ['ENSG00000148795', 'ENSG00000165359', 'ENSG00000150676']
ginfo = mg.querymany(ens, scopes='ensembl.gene')

for g in ginfo:
    for k, v in g.items():
        print("- {0: <10}: {1}".format(k, v))
    print('')

querying 1-3...done.
Finished.
- query     : ENSG00000148795
- _id       : 1586
- _score    : 18.763334
- entrezgene: 1586
- name      : cytochrome P450 family 17 subfamily A member 1
- symbol    : CYP17A1
- taxid     : 9606

- query     : ENSG00000165359
- _id       : 203522
- _score    : 19.774754
- entrezgene: 203522
- name      : integrator complex subunit 6 like
- symbol    : INTS6L
- taxid     : 9606

- query     : ENSG00000150676
- _id       : 220047
- _score    : 21.06969
- entrezgene: 220047
- name      : coiled-coil domain containing 83
- symbol    : CCDC83
- taxid     : 9606



## Merge duplicated Entrez_GENE_ID

In [180]:
id_list = []
entrez_id_list = df_results[df_results.index.str.contains('///')].index.tolist()
for entrez_id in entrez_id_list:
    entrez_ids = entrez_id.split(' /// ')
    id_list.extend(entrez_ids)

print(len(id_list))
id_list

3871


['780',
 '100616237',
 '7318',
 '100847079',
 '2342',
 '100529261',
 '6158',
 '102465483',
 '6144',
 '26771',
 '619499',
 '100131205',
 '3178',
 '664709',
 '6207',
 '85388',
 '100508408',
 '6205',
 '84546',
 '1655',
 '100616387',
 '100616408',
 '6139',
 '26790',
 '26791',
 '497661',
 '100124516',
 '100526842',
 '7919',
 '692199',
 '100532737',
 '8683',
 '283459',
 '23020',
 '101929240',
 '375',
 '100500810',
 '3020',
 '3021',
 '440926',
 '6124',
 '595097',
 '595098',
 '595099',
 '595100',
 '6189',
 '8944',
 '7184',
 '100500842',
 '801',
 '805',
 '808',
 '9782',
 '724102',
 '6418',
 '646817',
 '8663',
 '728689',
 '10399',
 '619570',
 '619571',
 '1937',
 '100500804',
 '1933',
 '619569',
 '2280',
 '101929368',
 '23521',
 '26816',
 '26817',
 '26818',
 '26819',
 '728658',
 '6427',
 '693221',
 '5757',
 '728026',
 '100302285',
 '100422872',
 '100422885',
 '100506248',
 '3303',
 '3304',
 '3305',
 '6202',
 '26811',
 '94163',
 '6142',
 '26780',
 '10528',
 '26792',
 '692201',
 '692213',
 '1003021

In [181]:
entrez_id_list

['780 /// 100616237',
 '7318 /// 100847079',
 '2342 /// 100529261',
 '6158 /// 102465483',
 '6144 /// 26771 /// 619499 /// 100131205',
 '3178 /// 664709',
 '6207 /// 85388 /// 100508408',
 '6205 /// 84546',
 '1655 /// 100616387 /// 100616408',
 '6139 /// 26790 /// 26791 /// 497661 /// 100124516 /// 100526842',
 '7919 /// 692199 /// 100532737',
 '8683 /// 283459',
 '23020 /// 101929240',
 '375 /// 100500810',
 '3020 /// 3021 /// 440926',
 '6124 /// 595097 /// 595098 /// 595099 /// 595100',
 '6189 /// 8944',
 '7184 /// 100500842',
 '801 /// 805 /// 808',
 '9782 /// 724102',
 '6418 /// 646817',
 '8663 /// 728689',
 '10399 /// 619570 /// 619571',
 '1937 /// 100500804',
 '1933 /// 619569',
 '2280 /// 101929368',
 '23521 /// 26816 /// 26817 /// 26818 /// 26819 /// 728658',
 '6427 /// 693221',
 '5757 /// 728026 /// 100302285 /// 100422872 /// 100422885 /// 100506248',
 '3303 /// 3304 /// 3305',
 '6202 /// 26811 /// 94163',
 '6142 /// 26780',
 '10528 /// 26792 /// 692201 /// 692213 /// 1003021

In [182]:
entrez_single_id_list = df_results[~df_results.index.str.contains('///')].index.tolist()
entrez_single_id_list

['5982',
 '3310',
 '7849',
 '2978',
 '7067',
 '11099',
 '6352',
 '1571',
 '2049',
 '2101',
 '1548',
 '2621',
 '4323',
 '8717',
 '5337',
 '441263',
 '572',
 '10594',
 '826',
 '11224',
 '1982',
 '8664',
 '11315',
 '6727',
 '2665',
 '6135',
 '377',
 '6152',
 '3183',
 '4735',
 '6233',
 '2197',
 '23435',
 '1072',
 '6141',
 '8665',
 '6193',
 '6155',
 '6164',
 '4677',
 '56910',
 '6143',
 '5250',
 '6133',
 '6128',
 '23399',
 '4736',
 '11335',
 '5690',
 '10657',
 '51493',
 '2079',
 '23',
 '1603',
 '7528',
 '10899',
 '11143',
 '7705',
 '9092',
 '3608',
 '9552',
 '8882',
 '6881',
 '10438',
 '4841',
 '387',
 '10921',
 '6229',
 '6156',
 '4869',
 '3326',
 '3550',
 '8724',
 '821',
 '9733',
 '27013',
 '10285',
 '4670',
 '3184',
 '9045',
 '2987',
 '79036',
 '4946',
 '533',
 '3735',
 '6194',
 '6201',
 '23326',
 '10944',
 '6923',
 '1327',
 '10959',
 '2339',
 '6230',
 '6167',
 '3094',
 '1938',
 '6204',
 '8992',
 '3190',
 '51433',
 '3192',
 '8661',
 '4478',
 '81',
 '351',
 '5573',
 '1832',
 '5885',
 '9948'

In [184]:
common_elements = list(set(entrez_single_id_list).intersection(set(id_list)))
print(len(common_elements))

905


In [185]:
print(len(common_elements))
print('id_list: {}, set: {}'.format(len(id_list),len(set(id_list))))
print('entrez_single_id_list: {}, set: {}'.format(len(entrez_single_id_list),len(set(entrez_single_id_list))))
print('entrez_id_list: {}, set: {}'.format(len(entrez_id_list),len(set(entrez_id_list))))
dups = [x for x in id_list if id_list.count(x) > 1]
print('dups: {}, set: {}'.format(len(dups),len(set(dups))))

905
id_list: 3871, set: 2884
entrez_single_id_list: 18229, set: 18229
entrez_id_list: 1356, set: 1356
dups: 1527, set: 540


In [194]:
full_entre_id_sets = []
cnt = 0
entrez_dups_list = []
for dup_id in set(dups):
    print(dup_id+':')
    id_set = []
    entrez_dups = []
    for multi_ids in entrez_id_list:
        if dup_id in multi_ids.split(' /// '):
            print('{}'.format(multi_ids))
            id_set.extend(multi_ids.split(' /// '))
            entrez_dups.append(multi_ids)
    id_set = list(set(id_set))
    id_set.sort(key=int)
    full_entre_id = ' /// '.join(id_set)
    print('Merged {}: {}\n'.format(dup_id,full_entre_id))
    full_entre_id_sets.append(full_entre_id)
    entrez_dups_list.append(entrez_dups)
    cnt+=1

entrez_dups_dict = dict(zip(full_entre_id_sets,entrez_dups_list))
full_entre_id_sets = list(set(full_entre_id_sets))

100101629:
2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399
2543 /// 2574 /// 26749 /// 645037 /// 645051 /// 729408 /// 729447 /// 100101629
2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399 /// 102724473
2543 /// 2574 /// 2575 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645037 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399
Merged 100101629: 2543 /// 2574 /// 2575 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645037 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 1

22947 /// 26582 /// 26583 /// 441056 /// 652301 /// 653543 /// 653544 /// 653545 /// 653548 /// 728410 /// 100133400 /// 100287823 /// 100288289 /// 100288687 /// 100291626 /// 100419743 /// 102723449
22947 /// 26583 /// 441056 /// 652301 /// 653543 /// 653544 /// 653545 /// 653548 /// 728410 /// 100288289 /// 100288687 /// 100291626 /// 100419743 /// 102723449
Merged 100288687: 22947 /// 26582 /// 26583 /// 441056 /// 652301 /// 653543 /// 653544 /// 653545 /// 653548 /// 728410 /// 100133400 /// 100287823 /// 100288289 /// 100288687 /// 100291626 /// 100419743 /// 102723449

25812:
25812 /// 29774 /// 29797
25812 /// 29774 /// 29797 /// 266697 /// 646074 /// 727983 /// 102725072 /// 102725315
Merged 25812: 25812 /// 29774 /// 29797 /// 266697 /// 646074 /// 727983 /// 102725072 /// 102725315

101927137:
7203 /// 101927137
478 /// 101927137
57604 /// 101927137
286336 /// 101927137
132320 /// 101927137
Merged 101927137: 478 /// 7203 /// 57604 /// 132320 /// 286336 /// 101927137

728127


101930404:
3653 /// 6638 /// 91380 /// 347746 /// 100033416 /// 100033433 /// 100033444 /// 100033450 /// 100033802 /// 100033820 /// 101930404
3653 /// 91380 /// 347746 /// 100033416 /// 100033433 /// 100033444 /// 100033450 /// 100033802 /// 100033820 /// 101930404
3653 /// 6638 /// 8926 /// 91380 /// 347746 /// 100033416 /// 100033433 /// 100033444 /// 100033450 /// 100033802 /// 100033820 /// 101930404
Merged 101930404: 3653 /// 6638 /// 8926 /// 91380 /// 347746 /// 100033416 /// 100033433 /// 100033444 /// 100033450 /// 100033802 /// 100033820 /// 101930404

401251:
4439 /// 401251 /// 100532732
401251 /// 100532732
Merged 401251: 4439 /// 401251 /// 100532732

653635:
374666 /// 375260 /// 653635 /// 100287171 /// 100288778 /// 101930154 /// 102465909 /// 102466751 /// 102723897
374666 /// 375260 /// 653635 /// 100287171 /// 100288778 /// 101930154 /// 102465909 /// 102466751
374666 /// 375260 /// 375690 /// 653635 /// 100134445 /// 100287171 /// 100288778 /// 100653296
Merged 

2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399
2543 /// 2574 /// 26749 /// 645037 /// 645051 /// 729408 /// 729447 /// 100101629
2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399 /// 102724473
2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 645073 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100132399
2543 /// 2574 /// 2575 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645037 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399
Merged 729447: 2543 /// 2574 /// 2575 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645037 /

In [199]:
entrez_dups_list = [x for l in entrez_dups_dict.values() for x in l]
print(len(entrez_dups_dict))
print(len(entrez_dups_list))
print(len(set(entrez_dups_list)))

228
678
446


In [200]:
entrez_dups_list

['2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399',
 '2543 /// 2574 /// 26749 /// 645037 /// 645051 /// 729408 /// 729447 /// 100101629',
 '2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399 /// 102724473',
 '2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 645073 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100132399',
 '2543 /// 2574 /// 2575 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645037 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399',
 '1485 /// 246100',
 '1485 /// 30848 /// 246100',
 '151230 /// 100526832',
 '151230 //

In [188]:
df_results.reset_index(inplace=True)
for merged_entrez_id, entrez_dups_list in entrez_dups_dict.items():
    # print(merged_entrez_id)
    # print(entrez_dups_list)
    df_results['ENTREZ_GENE_ID'].replace(to_replace=entrez_dups_list,
                                         value=merged_entrez_id,
                                         inplace=True)
#     for entrez_dups in entrez_dups_list:
#         df_results.index.replace(index={entrez_dups:merged_entrez_id},inplace=True)
df_results.set_index('ENTREZ_GENE_ID',inplace=True)
df_results

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3310,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7849,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7067,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6352,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
df_results.fillna(-1).groupby(level=0).max()

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
100,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1000,1.0,1.0,0.0,1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
100009676,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10001,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10002,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [191]:
# df_results[df_results.index.str.contains('///')]
df_results.dropna(how='all')

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3310,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7849,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7067,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6352,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
print(cnt)
print(len(entrez_dups_dict))
len(full_entre_id_sets)

540
276


276

In [92]:
import sys

from Bio import Entrez

# *Always* tell NCBI who you are
Entrez.email = "examples1@gmail.com"

def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene",id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        #FIXME: How generate NAs instead of causing an error with invalid IDs?
        print("An error occurred while retrieving the annotations.")
        print("The error returned was %s" % e)
        sys.exit(-1)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key =
            queryKey)
    annotations = Entrez.read(data)

    print("Retrieved %d annotations for %d genes" % (len(annotations),
            len(id_list)))

    return annotations

In [93]:
def print_data(annotation):
    for gene_data in annotation:
        gene_id = gene_data["Id"]
        gene_symbol = gene_data["NomenclatureSymbol"]
        gene_name = gene_data["Description"]
        print("ID: %s - Gene Symbol: %s - Gene Name: %s" % (gene_id, gene_symbol, gene_name))

In [ ]:
annotations =  retrieve_annotation(id_list[0:10])

In [107]:
# for key,annotation in annotations.items():
#     print_data(annotation)
print_data(annotations)

TypeError: string indices must be integers

In [106]:
annotation

DictElement({'DocumentSummary': [DictElement({'Name': 'DDR1', 'Description': 'discoidin domain receptor tyrosine kinase 1', 'Status': '0', 'CurrentID': '0', 'Chromosome': '6', 'GeneticSource': 'genomic', 'MapLocation': '6p21.33', 'OtherAliases': 'CAK, CD167, DDR, EDDR1, HGK2, MCK10, NEP, NTRK4, PTK3, PTK3A, RTK6, TRKE', 'OtherDesignations': 'epithelial discoidin domain-containing receptor 1|CD167 antigen-like family member A|PTK3A protein tyrosine kinase 3A|cell adhesion kinase|mammary carcinoma kinase 10|neuroepithelial tyrosine kinase|neurotrophic tyrosine kinase, receptor, type 4|protein-tyrosine kinase RTK-6|tyrosine kinase DDR|tyrosine-protein kinase CAK', 'NomenclatureSymbol': 'DDR1', 'NomenclatureName': 'discoidin domain receptor tyrosine kinase 1', 'NomenclatureStatus': 'Official', 'Mim': ['600408'], 'GenomicInfo': [{'ChrLoc': '6', 'ChrAccVer': 'NC_000006.12', 'ChrStart': '30880957', 'ChrStop': '30900155', 'ExonCount': '25'}], 'GeneWeight': '17493', 'Summary': 'Receptor tyrosin